<a href="https://colab.research.google.com/github/shakib-sadat/Bangla_Sentiment_Analysis_Scikit_LLM/blob/main/Copy_of_LLM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
# !pip install -U pip
!pip install accelerate
!pip install appdirs
!pip install bitsandbytes
!pip install datasets
!pip install fire
# !pip install git+https://github.com/huggingface/peft.git
# !pip install git+https://github.com/huggingface/transformers.git
!pip install torch
!pip install sentencepiece
!pip install tensorboardX
!pip install gradio

In [ ]:
!pip install peft

In [ ]:
!pip install transformers

In [ ]:
import transformers
import textwrap
from transformers import LlamaTokenizer, LlamaForCausalLM
import os
import sys
from typing import List

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)

import fire
import torch
from datasets import load_dataset
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pylab import rcParams
import json

%matplotlib inline
sns.set(rc={'figure.figsize':(8, 6)})
sns.set(rc={'figure.dpi':100})
sns.set(style='white', palette='muted', font_scale=1.2)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel("clean_reviews_multi.xlsx")

In [ ]:
df.head()

In [ ]:
df.Comment_Type.value_counts()

In [ ]:
df.shape

In [ ]:
df.Comment_Type.value_counts().plot(kind='bar');

In [ ]:
dataset_data = [
    {
        "instruction": "Detect the sentiment of the Comment.",
        "input": row_dict["cleaned"],
        "output": row_dict["Comment_Type"]
    }
    for row_dict in df.to_dict(orient="records")
]

In [ ]:
dataset_data[0]

{'instruction': 'Detect the sentiment of the Comment.',
 'input': 'খুবই আরাম দায়ক শীতের ঘরে পরার পারফেক্ট সেন্ডেল জুতা ধন্যবাদ সেলারকে এটার প্রাইস আরো কমানো',
 'output': 'Positive'}

In [ ]:
with open("alpaca-sentiment-dataset.json", "w") as f:
   json.dump(dataset_data, f)

In [ ]:
BASE_MODEL = "decapoda-research/llama-7b-hf"

model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

In [ ]:
data = load_dataset("json", data_files="alpaca-sentiment-dataset.json")

In [ ]:
data["train"]

In [ ]:
CUTOFF_LEN = 256

In [ ]:
def generate_prompt(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""

In [ ]:
def tokenize(prompt, add_eos_token=True):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt

In [ ]:
train_val = data["train"].train_test_split(
    test_size=200, shuffle=True, seed=42
)
train_data = (
    train_val["train"].shuffle().map(generate_and_tokenize_prompt)
)
val_data = (
    train_val["test"].shuffle().map(generate_and_tokenize_prompt)
)

In [ ]:
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]

BATCH_SIZE = 128
MICRO_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 30
OUTPUT_DIR = "experiments"

In [ ]:
model = prepare_model_for_int8_training(model)
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

In [ ]:
training_arguments = transformers.TrainingArguments(
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    warmup_steps=20,
    max_steps=TRAIN_STEPS,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=10,
    optim="adamw_torch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    eval_steps=5,
    save_steps=5,
    output_dir=OUTPUT_DIR,
    save_total_limit=3,
    metric_for_best_model = "eval_accuracy",
    load_best_model_at_end=True,
    report_to="tensorboard"
)

In [ ]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True,
    label_pad_token_id=tokenizer.pad_token_id, # Add this line
)

In [ ]:
import torch
from transformers import Trainer, DataCollator
from sklearn.metrics import accuracy_score

In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions.argmax(-1)
    accuracy = (pred_ids == labels_ids).mean()
    return {"accuracy": accuracy}

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_arguments,
    data_collator=data_collator,
    compute_metrics=compute_metrics,  # Add this line
)
model.config.use_cache = False
old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(
        self, old_state_dict()
    )
).__get__(model, type(model))

model = torch.compile(model)

trainer.train()

model.save_pretrained(OUTPUT_DIR)

In [ ]:
result = trainer.evaluate()
accuracy = result["eval_accuracy"]
print("Accuracy:", accuracy)